![](https://i.imgur.com/qkg2E2D.png)

# UnSupervised Learning Methods

## Exercise 002 - Part IV

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 03/04/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/UnSupervisedLearningMethods/2023_03/Exercise0002Part004.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp

# Machine Learning
from sklearn import datasets
from sklearn.cluster import AgglomerativeClustering

# Computer Vision

# Statistics

# Miscellaneous
import os
import math
from platform import python_version
import random
import time
import urllib.request

# Typing
from typing import Callable, List, Tuple, Union

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout
from sklearn.metrics import euclidean_distances
from sklearn.neighbors import NearestNeighbors

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

DATA_FILE_URL   = r'https://drive.google.com/uc?export=download&confirm=9iBg&id=11YqtdWwZSNE-0KxWAf1ZPINi9-ar56Na'
DATA_FILE_NAME  = r'ClusteringData.npy'


## Guidelines

 - Fill the full names and ID's of the team members in the `Team Members` section.
 - Answer all questions / tasks within the Jupyter Notebook.
 - Use MarkDown + MathJaX + Code to answer.
 - Verify the rendering on VS Code.
 - Submission in groups (Single submission per group).
 - You may and _should_ use the forums for questions.
 - Good Luck!

* <font color='brown'>(**#**)</font> The `Import Packages` section above imports most needed tools to apply the work. Please use it.
* <font color='brown'>(**#**)</font> You may replace the suggested functions to use with functions from other packages.
* <font color='brown'>(**#**)</font> Whatever not said explicitly to implement maybe used by a 3rd party packages.

## Team Members

- `Nadav_Talmon_203663950`.
- `Nadav_Shaked_312494925`.
- `Adi_Rosenthal_316550797`.

## Generate / Load Data

In [ ]:
# Download Data
# This section downloads data from the given URL if needed.

if not os.path.exists(DATA_FILE_NAME):
    urllib.request.urlretrieve(DATA_FILE_URL, DATA_FILE_NAME)

In [ ]:
# Generate / Load Data

numSamples  = 1000
mA          =  np.array([[0.6, -0.6], [-0.4, 0.8]])

mX1 = datasets.make_circles(n_samples = numSamples, noise = 0.02)[0]
mX2 = datasets.make_moons(n_samples = numSamples, noise = 0.05)[0]
mX3 = datasets.make_blobs(n_samples = numSamples, random_state = 170)[0] @ mA
mX4 = datasets.make_blobs(n_samples = numSamples, random_state = 170, cluster_std = [0.8, 2, 0.4])[0] 
mX5 = np.load(DATA_FILE_NAME)

lDataSet = [mX1, mX2, mX3, mX4, mX5]
numDataSets = len(lDataSet)

In [ ]:
# Plot Data
hF, hAs = plt.subplots(nrows = 1, ncols = numDataSets, figsize = (18, 5))
hAs = hAs.flat

for ii, hA in enumerate(hAs):
    mX = lDataSet[ii]
    hA.scatter(mX[:, 0], mX[:, 1], c = 'lime', s = 15, edgecolor = 'k')
    hA.axis('equal')
    
plt.tight_layout()
plt.show()

## 8. Clustering by Density based Spatial Clustering of Applications with Noise (DBSCAN)

### 8.1. DBSCAN Algorithm

In this section we'll implement the DBSCAN algorithm:

1. Implement an auxiliary function to compute the connected components (`GetConnectedComponents()`).  
   You may choose any implementation strategy (`DFS` / `BFS`, ect...).
2. Implement the function `DBSCAN()`.  
   The function should label noise points as `-1`.

* <font color='brown'>(**#**)</font> Implementation should be efficient (Memory and operations). Total run time expected to be **less than 20 seconds**.


In [ ]:
def GetConnectedComponents(mG: np.ndarray, core_points: np.ndarray) -> np.ndarray:
    '''
    Extract the connected components of a graph.
    Args:
        mG          - Graph matrix.
    Output:
        vL          - Label per component.
    Remarks:
        - This is a !!BFS / DFS!! implementation.
    '''

    def dfs(node, component):
        stack = [node]
        while stack:
            n = stack.pop()
            if vL[n] == 0:
                vL[n] = component
                if core_points[n]:
                    neighbors = np.where(mG[n] != 0)[0]
                    stack.extend(neighbors)

    N = mG.shape[0]
    vL = np.zeros(N)  # Component labels
    component = 1  # Current component label

    for i in range(N):
        if core_points[i] and vL[i] == 0:
            dfs(i, component)
            component += 1

    return vL.astype(int)

In [ ]:
def DBSCAN(mX: np.ndarray, Z: int, r: float) -> np.ndarray:
    '''
    DBSCAN Algorithm.
    Args:
        mX  - Input data with shape N x d.
        Z   - Number of points required to be a core point.
        r   - Neighborhood radius.
    Output:
        vL  - The labels (-1, 0, 1, .., K - 1) per sample with shape (N, ).
    Remarks:
        - Clusters will have the labels {0, 1, ..., K - 1}.
        - Noise samples will have the label `-1`.
    '''

    # Step 1: Find core points
    mD = euclidean_distances(mX)  # Pairwise distance matrix
    core_points = np.sum(mD <= r, axis=1) >= Z

    # Step 2: Build the graph
    mG = np.where(mD <= r, 1, 0)  # Adjacency matrix

    # Step 3: Find connected components
    labels = GetConnectedComponents(mG, core_points)  # Connected components

    return labels

### 8.2. Clustering the Data Set

In this section we'll use the implementation of the DSCAN algorithm.
The tasks are:

1. Use the data set `mX4`.
2. Tweak the parameters until you have 3 clusters.
3. Display results.

In [ ]:
#===========================Fill This===========================#
# 1. Set parameters.
# 2. Apply the algorithm.

Z = 5
r = 0.688
labels = DBSCAN(mX4, Z, r)
label_unique = np.sort(np.unique(labels))

#===============================================================#

In [ ]:
#===========================Fill This===========================#
# 1. Plot the clustered data.
# !! The noise samples should also be labeled.

fig, ax = plt.subplots()
unique_labels = set(labels) - {-1}
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

noise_samples = mX4[labels == 0]

for label, color in zip(unique_labels, colors):
    if label == 0:
        continue  # Skip noise samples for now
    cluster_samples = mX4[labels == label]
    ax.scatter(cluster_samples[:, 0], cluster_samples[:, 1], color=color, label=f'Cluster {label}')

ax.scatter(noise_samples[:, 0], noise_samples[:, 1], color='black', label='Noise', marker='x')

ax.legend()
ax.set_title('DBSCAN Clustering')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

#===============================================================#

### 8.3. An Algorithm to Set the Parameters Automatically Given a Data Set

Can you think about an algorithm to automatically infer optimal parameters of the DBSCAN algorithm given a data set?   

1. Sketch the algorithm (Words / Diagram).
2. Implement and test on `mX4`.
3. Plot the results.

* <font color='brown'>(**#**)</font> Run time should be reasonable (Single number of seconds).
* <font color='brown'>(**#**)</font> Good answers might be given a bonus points of up to 4 points.

### 8.3. Solution

The algorithm uses a systematic approach to find the best r value for each num_points by idea that the k-nearest neighbor (KNN) distance can represent the density of the data.

The algorithm evaluates different combinations of r and num_points by utilizing the Silhouette Coefficient. This coefficient measures the fit of each sample within its assigned cluster by comparing the average distance to other samples in the same cluster with the average distance to samples in the nearest neighboring cluster.

---

In [ ]:
#===========================Fill This===========================#
# Implement a function which gets a data set and output the `Z` and `r` parameters of `DBSCAN()`.

from sklearn.metrics import silhouette_score

def get_dbscan_parameters(data):
    best_score = 0
    best_params = {}
    labels_max = None
    for num_points in np.arange(2, 20):
        nbrs = NearestNeighbors(n_neighbors=num_points).fit(data)

        distances, indices = nbrs.kneighbors(data)

        distances = np.sort(distances, axis=0)
        distances = distances[:, num_points - 1]
        for ratio in np.arange(0.75, 1.0, 0.05):
            r = distances[int(len(distances) * ratio)]
            labels = DBSCAN(data, num_points, r)
            # The Silhouette Coefficient - calc mean intra-cluster distance (`a`) and the mean nearest-cluster distance (``b``) for each
            # sample.  The Silhouette Coefficient for a sample is ``(b - a) / max(a,b)``.
            if len(np.unique(labels)) > 1:
                score = silhouette_score(data, labels)
            else:
                score = 0
            if score > best_score:
                best_score = score
                labels_max = labels
                best_params = {'r': r, 'num_points': num_points}
    return best_params["r"], best_params["num_points"]
    
#===============================================================#

In [ ]:
#===========================Fill This===========================#
# Test your algorithm on `mX4` data set. Show results.

data = mX4
r, Z = get_dbscan_parameters(data)
print(f"r: {r}")
print(f"Z: {Z}")

labels = DBSCAN(data, Z, r)
label_unique = np.sort(np.unique(labels))

fig, ax = plt.subplots()

unique_labels = set(labels) - {-1}
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

noise_samples = data[labels == 0]

for label, color in zip(unique_labels, colors):
    if label == 0:
        continue  # Skip noise samples for now
    cluster_samples = data[labels == label]
    ax.scatter(cluster_samples[:, 0], cluster_samples[:, 1], color=color, label=f'Cluster {label}')

ax.scatter(noise_samples[:, 0], noise_samples[:, 1], color='black', label='Noise', marker='x')

ax.legend()
ax.set_title('DBSCAN Clustering')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()

#===============================================================#

> Royi: <font color='green'>✓ +2 Points (Bonus)</font>. 

### 8.4. Test Methods on the Data Set

In this section we'll compare 4 methods on each data set.  
The 4th methods is `AgglomerativeClustering` which is imported from `SciKit Learn`.

1. Run each method on each data set.
2. Plot a grid of results (Using `plt.subplots()`): Each row is a different method, each column is a different data set.
3. Optimize the parameters per data set per method.

The final result is a grid of `4 x 5` scatter plots.

* <font color='brown'>(**#**)</font> You should use `CourseAuxFun.py` and import your self implemented functions from the module.

In [ ]:
#===========================Fill This===========================#
# Display the results of each method

from CourseAuxFun_GRP_F import *

def KMeans_lambda(data, K):
    k_plus_plus = 1
    mC = InitKMeans(data, K, k_plus_plus)
    mC, vL, lO = KMeans(data, mC, 1000, 1e-5)
    return vL

def SoftGMM_lambda(data, K):
    mμ, tΣ, vW = InitGmm(data, K)
    initialize = mμ.copy()
    mμ, tΣ, vW, vL, lO = GMM(data, initialize, tΣ, vW, 1000, 1e-5)
    return vL

methods = [{
                'method_name': 'K-means',
                'has_noise_points': False,
                'function': lambda dataset, n_clusters, _, __: KMeans_lambda(dataset, n_clusters),
            },
            {
                'method_name': 'Soft GMM',
                'has_noise_points': False,
                'function': lambda dataset, n_clusters, _, __: SoftGMM_lambda(dataset, n_clusters),
            },
            {
                'method_name': 'DBScan',
                'has_noise_points': True,
                'function': lambda dataset, n_clusters, Z, r: DBSCAN(dataset, Z, r),
            },
            {
                'method_name': 'Agglomerative',
                'has_noise_points': False,
                'function': lambda dataset, n_clusters, _, __: AgglomerativeClustering(n_clusters).fit(dataset).labels_,
            }]

datasets = [{
                'dataset_name': 'mX1',
                'dataset': mX1,
                'optimal_cluster_number': 2,
                'optimal_Z': 4,
                'optimal_r': 0.06,
            },
            {
                'dataset_name': 'mX2',
                'dataset': mX2,
                'optimal_cluster_number': 2,
                'optimal_Z': 4,
                'optimal_r': 0.09,
            },
            {
                'dataset_name': 'mX3',
                'dataset': mX3,
                'optimal_cluster_number': 3,
                'optimal_Z': 5,
                'optimal_r': 0.35,
            },
            {
                'dataset_name': 'mX4',
                'dataset': mX4,
                'optimal_cluster_number': 3,
                'optimal_Z': 5,
                'optimal_r': 0.688,
            },
            {
                'dataset_name': 'mX5',
                'dataset': mX5,
                'optimal_cluster_number': 2,
                'optimal_Z': 8,
                'optimal_r': 0.038,
            }]

results = np.random.rand(len(methods), len(datasets))  # Replace with your actual results

# Step 2: Plot a grid of scatter plots
fig, axes = plt.subplots(nrows=len(methods), ncols=len(datasets), figsize=(12, 10))

for i, method in enumerate(methods):
    for j, dataset in enumerate(datasets):
        ax = axes[i][j]
        mX = dataset['dataset']

        labels = method['function'](dataset['dataset'], dataset['optimal_cluster_number'], dataset['optimal_Z'], dataset['optimal_r'])
        unique_labels = set(labels) - {-1}

        colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

        for label, color in zip(unique_labels, colors):
            if method['has_noise_points'] and label == 0:
                noise_samples = mX[labels == 0]
                ax.scatter(noise_samples[:, 0], noise_samples[:, 1], c='black', label='Noise', marker='x')
            else:
                cluster_samples = mX[labels == label]
                ax.scatter(cluster_samples[:, 0], cluster_samples[:, 1], c=color, label=f'Cluster {label}')

        ax.set_title(f'{method["method_name"]} on {dataset["dataset_name"]}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

fig.suptitle('Clustering Results', fontsize=16)
plt.tight_layout()
plt.show()

#===============================================================#

> Royi: ❌, Exaggerated run time.  
> Royi: <font color='red'>-4</font>.

![](https://i.imgur.com/Y8JeBhi.png)